In [ ]:
import subprocess
import os
import signal

# Start the usb_cam node in the background
usb_cam_process = subprocess.Popen(
    ['ros2', 'run', 'usb_cam', 'usb_cam_node_exe'],
    stdout=subprocess.PIPE,
    stderr=subprocess.PIPE,
    preexec_fn=os.setsid # Groups the process for easier termination
)

print("usb_cam node started.")

In [ ]:
# This cell sets up the ROS 2 camera subscriber and the Jupyter interface for data collection with XY clicks.

import rclpy
from rclpy.node import Node
from sensor_msgs.msg import Image
from cv_bridge import CvBridge

import ipywidgets as widgets
from IPython.display import display
from jupyter_clickable_image_widget import ClickableImageWidget

from uuid import uuid1
import os
import glob
import cv2
import numpy as np
import threading

# Directory where the dataset will be stored
DATASET_DIR = 'dataset_xy'
os.makedirs(DATASET_DIR, exist_ok=True)

# CV bridge to convert ROS images to OpenCV format
bridge = CvBridge()

# Interface widgets
camera_widget = ClickableImageWidget(width=640, height=480)
snapshot_widget = widgets.Image(width=640, height=480)
count_widget = widgets.IntText(description='count', value=len(glob.glob(os.path.join(DATASET_DIR, '*.jpg'))))

# Global state variables
camera_node = None
camera_active = False
ros_thread = None

# Define the camera subscriber node
class CameraNode(Node):
    def __init__(self):
        super().__init__('jupyter_camera_listener')
        self.subscription = self.create_subscription(
            Image,
            '/image_raw',
            self.listener_callback,
            10
        )

    def listener_callback(self, msg):
        global camera_active
        if not camera_active:
            return

        cv_image = bridge.imgmsg_to_cv2(msg, desired_encoding='bgr8')
        jpeg = cv2.imencode('.jpg', cv_image)[1].tobytes()
        camera_widget.value = jpeg

# Function to start the ROS camera listener in a background thread
def start_ros_camera():
    global camera_node, camera_active, ros_thread

    if camera_active or rclpy.ok():
        print("Camera is already active or ROS 2 is still running.")
        return

    def ros_spin():
        global camera_node
        rclpy.init()
        camera_node = CameraNode()
        rclpy.spin(camera_node)
        if camera_node is not None:
            camera_node.destroy_node()
        rclpy.shutdown()
        print("🔁 ROS 2 detenido correctamente.")

    ros_thread = threading.Thread(target=ros_spin, daemon=True)
    ros_thread.start()
    camera_active = True
    print("Camera started.")

# Function to stop the ROS camera listener and the usb_cam node
def stop_ros_camera():
    global camera_active, camera_node
    camera_active = False

    # Kill the usb_cam process group
    os.killpg(os.getpgid(usb_cam_process.pid), signal.SIGINT)
    print("Camera deactivated. Waiting for ROS thread to finish...")

    if camera_node:
        camera_node.destroy_node()
        camera_node = None

# Start/stop buttons
start_button = widgets.Button(description='Iniciar Cámara', button_style='success')
stop_button = widgets.Button(description='Apagar Cámara', button_style='danger')

start_button.on_click(lambda b: start_ros_camera())
stop_button.on_click(lambda b: stop_ros_camera())

# Function to save image on click
def save_snapshot(_, content, msg):
    if not camera_active:
        print("Camera is not active. Snapshot not saved.")
        return

    if content['event'] == 'click':
        data = content['eventData']
        x = int(data['offsetX'])
        y = int(data['offsetY'])

        uuid_str = 'xy_%03d_%03d_%s' % (x, y, uuid1())
        image_path = os.path.join(DATASET_DIR, uuid_str + '.jpg')

        current_image = cv2.imdecode(np.frombuffer(camera_widget.value, dtype=np.uint8), cv2.IMREAD_COLOR)
        if current_image is not None:
            cv2.imwrite(image_path, current_image)

            annotated = current_image.copy()
            cv2.circle(annotated, (x, y), 8, (0, 255, 0), 3)
            snapshot_widget.value = cv2.imencode('.jpg', annotated)[1].tobytes()

            count_widget.value = len(glob.glob(os.path.join(DATASET_DIR, '*.jpg')))

camera_widget.on_msg(save_snapshot)

# Assemble and display the interface
data_collection_widget = widgets.VBox([
    widgets.HBox([camera_widget, snapshot_widget]),
    count_widget,
    widgets.HBox([start_button, stop_button])
])

display(data_collection_widget)
